In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
import re

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv("test.csv")
df_train

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
5,569677636613439488,negative,United,NaN,bobgiolito,NaN,0,@united Why did you load us in this flying sar...,NaN,2015-02-22 17:58:27 -0800,"Los Angeles, CA",Pacific Time (US & Canada)
6,569658903044218880,negative,Delta,NaN,aaronkinnari,NaN,0,@JetBlue that is a stock response. Delays not ...,NaN,2015-02-22 16:44:00 -0800,Gotham,Quito
7,568542766860541952,positive,Delta,NaN,TimothySays,NaN,0,@JetBlue That'd be nice! Hoping to rack up eno...,NaN,2015-02-19 14:48:53 -0800,"Burlington, MA",Eastern Time (US & Canada)
8,570116209263427584,negative,United,NaN,lindaSWC,NaN,1,@united frankly worse customer service ever. P...,NaN,2015-02-23 23:01:11 -0800,NaN,NaN
9,568870144891600896,positive,Southwest,NaN,amyums,NaN,0,@SouthwestAir yeah haha. Never been in one. It...,NaN,2015-02-20 12:29:46 -0800,NaN,Central Time (US & Canada)


In [3]:
trainX = df_train['text']
trainY = df_train['airline_sentiment']
testX = df_test['text']
trainX.shape,trainY.shape,len(trainX)

((10980,), (10980,), 10980)

In [4]:
all_tweets = []
for i in range(len(trainX)):
    required = (word_tokenize(trainX.iloc[i]),trainY.iloc[i])
    all_tweets.append(required)

In [5]:
all_test_tweets = []
for i in range(len(testX)):
    jucked = (word_tokenize(testX.iloc[i]))
    all_test_tweets.append(jucked)

In [6]:
import random
random.shuffle(all_tweets)

In [7]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lmt = WordNetLemmatizer()
from nltk import pos_tag
import string

In [8]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [9]:
stops = stopwords.words('english')
stops = stops + list(string.punctuation)

In [10]:
def clean_tweet(words):
    after_clean_words = []
    for word in words:
        if word.lower() not in stops:
            pos = pos_tag([word])
            clean_word = lmt.lemmatize(word,pos=get_simple_pos(pos[0][1]))
            if not bool(re.search('\d',clean_word)):
                if len(clean_word)>2:
                    after_clean_words.append(clean_word)
    return after_clean_words 

In [11]:
all_tweets = [(clean_tweet(tweet),sentiment) for tweet,sentiment in all_tweets]

In [12]:
all_test_tweets = [clean_tweet(tweet) for tweet in all_test_tweets]

In [13]:
def split(all_docs):
    train_tweets = all_docs[0:10000]
    test_tweets = all_docs[10000:]
    return train_tweets,test_tweets

In [14]:
train_tweets,test_tweets = split(all_tweets)
len(train_tweets),len(test_tweets)

(10000, 980)

In [15]:
test_tweets[0]

(['SouthwestAir',
  'Worst',
  'customer',
  'service',
  'ever',
  'Cancelled',
  'Flightled',
  'connect',
  'flight',
  'hour',
  'ago',
  'amp',
  "n't",
  'tell',
  'stuck',
  'Tampa'],
 'negative')

In [16]:
def create_vocab(training_set):
    all_words = []
    for tw in training_set:
        all_words+=tw[0]
    freq = nltk.FreqDist(all_words)
    comm = freq.most_common(7000)
    features =  [i[0] for i in comm]
    return features

In [17]:
features = create_vocab(train_tweets)

In [18]:
train_tweets[0][1]

'negative'

In [19]:
def create_dataset_required(train_points,my_vocab):
    required_dataset = []
    for point in train_points:
        dict1 = {}
        for voc in my_vocab:
            dict1[voc] = point[0].count(voc)
        reqd_tuple = (dict1,point[1])
        required_dataset.append(reqd_tuple)
    return required_dataset
            

In [20]:
dataset_for_clf = create_dataset_required(train_tweets,features)

In [21]:
from nltk import NaiveBayesClassifier

In [22]:
clf =  NaiveBayesClassifier.train(dataset_for_clf)

In [23]:
testing_data_required = create_dataset_required(test_tweets,features)

In [24]:
nltk.classify.accuracy(clf,testing_data_required)

0.7887755102040817

In [25]:
clf.show_most_informative_features(20)

Most Informative Features
                favorite = 1              positi : negati =     34.1 : 1.0
                   Great = 1              positi : neutra =     33.0 : 1.0
                   THANK = 1              positi : negati =     29.0 : 1.0
                   thank = 1              positi : negati =     25.4 : 1.0
                 awesome = 1              positi : negati =     24.3 : 1.0
                    http = 2              neutra : negati =     24.1 : 1.0
               fantastic = 1              positi : negati =     24.0 : 1.0
                 helpful = 1              positi : neutra =     22.7 : 1.0
               wonderful = 1              positi : negati =     22.0 : 1.0
             outstanding = 1              positi : negati =     21.5 : 1.0
               excellent = 1              positi : negati =     20.4 : 1.0
                   great = 1              positi : neutra =     19.6 : 1.0
                   Thank = 1              positi : negati =     19.4 : 1.0

In [48]:
def create_dataset_required_for_test(test_points,my_vocab_test):
    required_dataset = []
    for point in test_points:
        dict1 = {}
        for voc in my_vocab_test:
            dict1[voc] = point.count(voc)
        required_dataset.append(dict1)
    return required_dataset         

In [49]:
output_csv_res = create_dataset_required_for_test(all_test_tweets,features)

In [50]:
op_csv = clf.classify_many(output_csv_res)

In [51]:
np_arr = np.array(op_csv)

In [52]:
df_cs = pd.DataFrame(np_arr)

In [53]:
df_cs.to_csv("yolo.csv",header=None)

In [26]:
from nltk.classify.scikitlearn import SklearnClassifier

In [27]:
from sklearn.svm import SVC

In [28]:
svc = SVC(gamma=0.0005,C=100)

In [29]:
clf2 = SklearnClassifier(svc)

In [ ]:
clf2.train(dataset_for_clf)

In [46]:
nltk.classify.accuracy(clf2,testing_data_required)

0.7918367346938775